In [11]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import urllib.parse
import ssl
import requests


# SSL 에러 방지
context = ssl._create_unverified_context()

# 기존 데이터셋 로드
data = pd.read_csv('Guro.csv', encoding='cp949')

# 맛집 정보를 담을 새로운 열 추가
data['맛집정보'] = ''

# 관광지명만 추출
tourist_spots = data['관광지명'].tolist()

# 관광지명을 URL에 적합하도록 인코딩
encoded_list = [urllib.parse.quote(s) for s in tourist_spots]

# 맛집 정보 가져오기
for spot, row_index in zip(encoded_list, range(len(encoded_list))):
    basic = f'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query={spot}+%EB%A7%9B%EC%A7%91'
    # ' 근처 숙박' -> %20%EA%B7%BC%EC%B2%98%20%EC%88%99%EB%B0%95
    url = requests.get(basic, headers={'User-Agent':'Mozilla/5.0'})
    html = urlopen(basic, context=context).read()
    soup = BeautifulSoup(html, 'html.parser')

    # 맛집명 + 카테고리 추출
    span_name = soup.find_all(class_='TYaxT')
    span_category = soup.find_all(class_='KCMnt')

    # 각 맛집명과 카테고리를 묶어서 맛집 정보 생성
    restaurant_info = []
    for name, category in zip(span_name, span_category):
        restaurant_info.append((name.text.strip(), category.text.strip()))

    # 데이터프레임에 맛집 정보 저장
    data.at[row_index, '맛집정보'] = restaurant_info

# 결과 출력
filtered_data = data[data['맛집정보'].apply(lambda x: len(x) > 0)]
filtered_data.head()

,시군구명,시도명,관광지명,Unnamed: 3,KTO카테고리대분류명,KTO카테고리중분류명,KTO카테고리소분류명,방문자수,맛집정보
0,구로구,서울특별시,푸른수목원&항동철길,https://korean.visitkorea.or.kr/detail/ms_deta...,자연,자연관광지,수목원,4277,"[(12NOI, 카페,디저트)]"
1,구로구,서울특별시,구로 중앙유통단지,https://korean.visitkorea.or.kr/detail/ms_deta...,쇼핑,쇼핑,전문매장/상가,3098,"[(라도맨션, 돼지고기구이), (하나마토 신도림본점, 술집), (우월소곱창 신도림본..."
3,구로구,서울특별시,오류시장,https://korean.visitkorea.or.kr/detail/ms_deta...,쇼핑,쇼핑,상설시장,563,"[(한동길참치 오류동본점, 일식당), (블랑베이커리카페, 카페,디저트), (이자카야..."
5,구로구,서울특별시,고척근린시장,https://korean.visitkorea.or.kr/detail/ms_deta...,쇼핑,쇼핑,상설시장,376,"[(BBQ치킨 고척근린시장점, 치킨,닭강정), (쌈빠찹쌀강정 고척근린시장점, 치킨,..."
7,구로구,서울특별시,개봉중앙시장,https://korean.visitkorea.or.kr/detail/ms_deta...,쇼핑,쇼핑,상설시장,289,"[(명가민물장어, 장어,먹장어요리), (양꼬치소년, 양꼬치), (메기 이자카야, 이..."


In [12]:
# 데이터프레임을 새로운 CSV 파일로 저장
filtered_data = filtered_data[(filtered_data['KTO카테고리대분류명'] != '음식') & (filtered_data['KTO카테고리소분류명'] != '관광호텔')]
filtered_data.to_csv('Guro_new.csv', index=False, encoding='utf-8-sig')